In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas.tseries import offsets
import random as rand
import matplotlib.pyplot as plot


# -----------------------
# Interview Q: 05/31/2023
    # Create a retention chart showing the new users and the percent of users still active

url = 'https://raw.githubusercontent.com/erood/interviewqs.com_code_snippets/master/Datasets/online_retail.csv'

data_0531 = pd.read_csv(url)

In [11]:
(data_0531
    .assign(
        real_date = lambda dfx: pd.to_datetime(dfx["InvoiceDate"]).dt.date, 
        month_end = lambda dfy: dfy["real_date"] + offsets.MonthEnd(), 
        signup_month = lambda dfz: (dfz.groupby("CustomerID")
            ["month_end"]
            .transform("min")), 
        periods_af = lambda dfa: (dfa["month_end"].dt.to_period("M").view(dtype='int64') - dfa["signup_month"].dt.to_period("M").view(dtype='int64'))
    )
    .groupby(["signup_month", "periods_af"])
    .agg(
        users = ("CustomerID", "nunique")
    )
    .reset_index()
    .assign(
        new_users = lambda dfb: (dfb.groupby("signup_month")
            ["users"]
            .transform("max"))
    )
    .query("periods_af > 0")
    .assign(retention = lambda dfc: dfc["users"] / dfc["new_users"])
    .pivot_table(
        values = "retention", 
        index = ["signup_month", "new_users"], 
        columns = "periods_af", 
    )
    .style.background_gradient(axis = 1)
    .applymap(lambda x: 'color: transparent' if np.isnan(x) else '')
)

,periods_af,1,2,3,4,5,6,7,8,9,10,11,12
signup_month,new_users,,,,,,,,,,,,
2010-12-31 00:00:00,948,0.367089,0.347046,0.381857,0.372363,0.387131,0.377637,0.357595,0.352321,0.399789,0.373418,0.498945,0.295359
2011-01-31 00:00:00,403,0.240695,0.275434,0.263027,0.325062,0.290323,0.260546,0.245658,0.312655,0.339950,0.362283,0.171216,nan
2011-02-28 00:00:00,374,0.235294,0.195187,0.262032,0.270053,0.256684,0.270053,0.275401,0.267380,0.304813,0.109626,nan,nan
2011-03-31 00:00:00,446,0.195067,0.257848,0.213004,0.224215,0.177130,0.264574,0.251121,0.286996,0.114350,nan,nan,nan
2011-04-30 00:00:00,317,0.217666,0.223975,0.220820,0.198738,0.249211,0.220820,0.264984,0.091483,nan,nan,nan,nan
2011-05-31 00:00:00,270,0.237037,0.185185,0.170370,0.211111,0.259259,0.251852,0.118519,nan,nan,nan,nan,nan
2011-06-30 00:00:00,238,0.189076,0.189076,0.260504,0.239496,0.340336,0.117647,nan,nan,nan,nan,nan,nan
2011-07-31 00:00:00,193,0.202073,0.212435,0.227979,0.274611,0.113990,nan,nan,nan,nan,nan,nan,nan
2011-08-31 00:00:00,166,0.259036,0.234940,0.259036,0.144578,nan,nan,nan,nan,nan,nan,nan,nan
